In [2]:
# 导入所需的函数
import os
import sys

# 获取当前 Notebook 的路径
notebook_path = os.getcwd()

print("当前 Notebook 的路径：", notebook_path)

# 获取项目根目录的路径
project_root = os.path.abspath(os.path.join(notebook_path, ".."))

# 切换当前工作目录到项目根目录
# 判断当前工作目录的后缀是否为"Pattern-Recognition"
if not notebook_path.endswith("Pattern-Recognition"):
    os.chdir(project_root)

# 将项目根目录添加到 Python 模块搜索路径
if project_root not in sys.path:
    sys.path.append(project_root)

print("当前工作目录：", os.getcwd())
print("当前的 Python 路径：")
print(sys.path)


from src.data_preprocessing import generate_synthetic_data, preprocess_data

当前 Notebook 的路径： d:\pycharm_workplace\Pattern-Recognition\notebooks
当前工作目录： d:\pycharm_workplace\Pattern-Recognition
当前的 Python 路径：
['d:\\Anaconda3\\python312.zip', 'd:\\Anaconda3\\DLLs', 'd:\\Anaconda3\\Lib', 'd:\\Anaconda3', '', 'd:\\Anaconda3\\Lib\\site-packages', 'd:\\Anaconda3\\Lib\\site-packages\\win32', 'd:\\Anaconda3\\Lib\\site-packages\\win32\\lib', 'd:\\Anaconda3\\Lib\\site-packages\\Pythonwin', 'd:\\Anaconda3\\Lib\\site-packages\\setuptools\\_vendor', 'd:\\pycharm_workplace\\Pattern-Recognition']


In [16]:
import pandas as pd

def sliding_window_analysis(
    df: pd.DataFrame, window_size=3, num_top_contents=50
) -> pd.DataFrame:
    """
    使用滑动窗口分析历史数据，识别当前的高频请求内容。

    本函数通过对每个内容ID的请求次数进行滑动窗口平均，选取最热门的前50个内容ID。

    :param df: 包含内容ID(content_id)、时间戳(timestamp)和请求次数(request_count)的DataFrame
    :param window_size: 滑动窗口的大小，默认为3
    :param num_top_contents: 选择最热门的内容数量，默认为50

    :return: 最受欢迎的内容ID列表
    """
    # 按内容ID分组
    # 使用 pivot_table 进行聚合，避免重复记录
    pivot_df = df.pivot_table(
        index="timestamp",
        columns="content_id",
        values="request_count",
        aggfunc="sum",
        fill_value=0,
    )

    print("pivot:")
    print(pivot_df.head())

    # 计算滑动窗口的总请求数
    rolling_sum = pivot_df.rolling(window=window_size).sum()

    # print("rolling_sum:")
    # print(rolling_sum.head())

    # 获取最新时间点的滑动窗口总请求数
    latest_window = rolling_sum.iloc[-1]

    print("latest_window:")
    print(latest_window)

    # 选择前N个高频内容
    top_contents = (
        latest_window.sort_values(ascending=False).head(num_top_contents).index.tolist()
    )

    high_freq_df = pd.DataFrame(
        {
            "content_id": top_contents,
            "total_requests": latest_window[top_contents].values,
        }
    )

    return high_freq_df

In [17]:
df_raw = generate_synthetic_data()
top_contents = sliding_window_analysis(df_raw)

print("处理后的数据：")
print(top_contents.head())

合成数据已保存到 D:\pycharm_workplace\Pattern-Recognition\data\raw
pivot:
content_id        1    2    3    4    5    6    7    8    9    10   ...  491  \
timestamp                                                           ...        
2024-12-01 00:00  140   76   41   29   32   23   22   18   15   16  ...    0   
2024-12-01 01:00  140   71   52   46   38   27   22   17   22   18  ...    0   
2024-12-01 02:00  160   82   41   52   29   30   12   20   13   20  ...    0   
2024-12-01 03:00  144   64   65   34   26   22   19   22   17   13  ...    0   
2024-12-01 04:00  171  100   52   34   23   24   19   25   16   12  ...    0   

content_id        492  493  494  495  496  497  498  499  500  
timestamp                                                      
2024-12-01 00:00    0    0    0    0    0    0    0    0    0  
2024-12-01 01:00    0    2    0    2    0    0    1    0    1  
2024-12-01 02:00    0    0    1    1    1    1    1    0    0  
2024-12-01 03:00    1    0    0    0    0    1    0  